# LMA: Levenberg-Marquardt Algorithm

The `LMA` operator implements the Levenberg-Marquardt algorithm, an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent.

It is an iterative algorithm. Every iteration, the residual and the Jacobian are calculated for a given set of parameters (the "current guess"). Then a new guess $p-\Delta p$ is calculated, such that:

$$(J^T J + \lambda D)\Delta p = J^T y$$

where $J$ is the Jacobian ($J_{ij}=\partial y_i / \partial p_j$), $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon, (J^T J)_{kk})$ with $\epsilon$ an arbitrarily small number, $y$ is the residual, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum numbers of iterations reached
* Residual below specified tolerance
* Relative change in parameters or residual below specified tolerance
* Maximum damping factor reached

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

## `LMA` Operator

    R←{X}f LMA Y

`f` is a configuration namespace or a function. Configuration namespaces can define the following configuration parameters (else, the provided defaults are used):

* `toli`: Maximum number of iterations (default `1E6`)
* `tols`: Tolerance for the sum of squared residuals (default `tolr`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `d=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)

With the exception of `toli` and `tols`, these parameters should be modified only by expert users.

Additionally, the namespace must contain two functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function must return the residual and Jacobian for the set of parameters given as right argument, and an optional left argument `X`.
In case `f` is a function, the default configuration is used, setting the `Eval` function to `f`.
The callback function is called every iteration before checking convergence, with a solution namespace as argument
and discarding the return value.

A solution namespace is a configuration namespace with the additional elements:

* `iter`: Number of iterations
* `sse`: Sum of squared residuals
* `rel`: Relative change metric
* `p0`: Initial guess
* `p`: Current guess

`Y` is a vector. If the depth of the vector is not larger than 1, it is enclosed first.
The first element of `Y` (or `⊂Y` if `1=≡⍵`) contains the initial guess for the parameters.
Additional elements of `Y` must be either configuration namespaces (overwriting the parameters at right with those at left) or a number indicating the value of the initial normalized damping factor.

The returned value `R` is a solution namespace.


## `LM` Operator

`LM` is a simplified version of `LMA`. Usage:

    R←X f LM g Y

where `f` is the evaluation function, `g` is the callback function (which takes as argument an `iter sse rel dnorm p` vector), `Y` is a two elements vector with the initial normalized damping factor and parameters guess. `X` is a vector defining the parameters `toli tols tolr tolg dini dinc ddec dmax dmin`. The return value `R` will be an `iter sse rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
]dinput
LM←{ti ts tr tg d0 di dd dx dn←⍺ ⋄ d p←⍵               ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢                     ⍝ damping factor normalization(λ)
    E←⍺⍺{s←+.×⍨⊃y j←⍺⍺ ⍵ ⋄ s ⍵ y j}                    ⍝ eval(p): sum(error²), parameters, residual, jacobian
    A←{s p y j←⍵ ⋄ s p,(t+.×j)(y+.×⍨t←⍉j)}             ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ d⊢←D ⍵                    ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←⎕CT⌈1 1⍉jj                  ⍝     change of parameters
        s0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        s1←s-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|s1÷s                      ⍝     relative change in parameters or residuals
        (⎕CT>s0)∧⎕CT<s1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤s0)∧tg≥s1÷s0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣s p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)s r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(ts>s)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ _←⍵⍵ i s r d p⊣s p jj jy←A E⊃⌽⍵            ⍝ init
    i s r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ return iterations, sse, change, norm damping, parameters
}

In [3]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ 0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('isrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'tols' 'ddec' 'dmin'                            ⍝ computed defaults
    nr←'iter' 'sse' 'rel' 'dnorm' 'p'                  ⍝ results
    c←(                                                ⍝ default config
        toli:1e6 ⋄ tolr:⎕CT ⋄ tolg:1e¯2                ⍝     tolerances
        dini:1e¯2 ⋄ dinc:5 ⋄ dmax:÷⎕CT                 ⍝     damping
        p0:p ⋄ verbose:0                               ⍝     initial parameters and logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵}            ⍝ format and print
    DF←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                        ⍝ display form and set results

    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←⎕NS c ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    _←c ⎕VSET(↑nc)(⎕VGET(↑nc)c.(tolr,÷dinc dmax))      ⍝ computed defaults
    c←⍺{(3=⎕NC'⍺⍺')∧0≠⍵.⎕NC'Left':⍵ ⋄ ⍵.Left←⍺⍺ ⋄ ⍵}c  ⍝ left arg
    CB←{c.CallBack c⎕VSET(↑nr)⍵}⊣P⍣(c.verbose≢0)       ⍝ callback function
    c⊣c.⎕DF DF(c⎕VGET↑n)c.(Left∘Eval)LM CB c.dnorm p   ⍝ return namespace
}

In [4]:
]link.export -overwrite # .

Exported: # → "C:\Users\34696\jupyter\lma"

## Examples

In [5]:
NOISY←{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}

### Exponential decay

In [6]:
ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵} ⋄ ExpDecEv←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
p←10 0.5 1 ⋄ y←0.25 NOISY p ExpDec x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p ExpDec x)(⊃x y ExpDecEv p)
1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT(x y∘ExpDecEv)LM⊢1(5 0.1 0.5) ⍝ ti ts tr tg d0 di dd dx dn
⊢s←x y ExpDecEv LMA(5 0.1 0.5)(verbose:1)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.04618 7.18745 4.75584 3.11226 2.38049 1.85042 1.47419 1.19638 1.14017 1.21392│
│ 11.00000 7.06531 4.67879 3.23130 2.35335 1.82085 1.49787 1.30197 1.18316 1.11109│
│ 0.04618 0.12214 0.07705 ¯0.11904 0.02713 0.02957 ¯0.02368 ¯0.10560 ¯0.04299 0.10283│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌─┬────────────┬───────────────┬──────────────┬─────────────────────────────────────┐
│7│0.0517214021│1.058007892E¯16│0.000063999999│10.10148479 0.4980300575 0.9783122596│
└─┴────────────┴───────────────┴──────────────┴─────────────────────────────────────┘

0 5.2297E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.5775E1 2.8029E¯1 1.0000E0 6.5375E0 3.8418E¯1 2.6531E0 
 2 3.0420E¯1 2.7309E¯1 2.0000E¯1 9.9061E0 5.5359E¯1 1.1507E0 
 3 5.9205E¯2 5.9773E¯4 4.0000E¯2 1.0081E1 4.9374E¯1 9.9139E¯1 
 4 5.1722E¯2 5.6509E¯6 8.0000E¯3 1.0101E1 4.9811E¯1 9.7874E¯1 
 5 5.1721E¯2 2.1831E¯9 1.6000E¯3 1.0101E1 4.9803E¯1 9.7831E¯1 
 6 5.1721E¯2 2.9992E¯13 3.2000E¯4 1.0101E1 4.9803E¯1 9.7831E¯1 
 7 5.1721E¯2 1.0580E¯16 6.4000E¯5 1.0101E1 4.9803E¯1 9.7831E¯1 
 s: 5.1721E¯2 p: 1.0101E1 4.9803E¯1 9.7831E¯1

In [7]:
⊢ed←x y ExpDecEv LMA⊂5 0.1 0.5
ed LMA ed.p0

s: 5.1721E¯2 p: 1.0101E1 4.9803E¯1 9.7831E¯1

s: 5.1721E¯2 p: 1.0101E1 4.9803E¯1 9.7831E¯1

### Linear system

In [8]:
Linear←{a b←⍺ ⋄ a+b×⍵} ⋄ LinearEv←{(y-⍵ Linear x)(⍉↑-(=⍨x)x)}
p←0.1 2 ⋄ y←1e¯3 NOISY p Linear x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Linear x)(⊃LinearEv p)
LinearEv LMA(0 0)(dinc:1e3 ⋄ verbose:1)

┌────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 0.09955 2.10001 4.09961 6.10025 8.09966 10.09988 12.09964 14.10045 16.09967 18.10002│
│ 0.10000 2.10000 4.10000 6.10000 8.10000 10.10000 12.10000 14.10000 16.10000 18.10000│
│ ¯0.00045 0.00001 ¯0.00039 0.00025 ¯0.00034 ¯0.00012 ¯0.00036 0.00045 ¯0.00033 0.00002│
└────────────────────────────────────────────────────────────────────────────────────────┘

0 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 1 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 2 1.1581E3 0.0000E0 1.0000E3 0.0000E0 0.0000E0 
 3 1.1581E3 0.0000E0 1.0000E6 0.0000E0 0.0000E0 
 4 1.1581E3 0.0000E0 1.0000E9 0.0000E0 0.0000E0 
 5 1.1581E3 3.4300E¯10 1.0001E12 9.0999E¯10 2.0158E¯10 
 6 1.1581E3 3.4300E¯7 1.0000E9 9.1090E¯7 2.0178E¯7 
 7 1.1577E3 3.4291E¯4 1.0000E6 9.1072E¯4 2.0174E¯4 
 8 8.4794E2 2.6756E¯1 1.0000E3 7.5750E¯1 1.7256E¯1 
 9 3.2734E¯1 9.9961E¯1 1.0000E0 3.8686E¯1 1.9370E0 
 10 7.6873E¯7 2.2139E¯2 1.0000E¯3 9.9774E¯2 2.0000E0 
 11 7.6749E¯7 1.0034E¯10 1.0000E¯6 9.9754E¯2 2.0000E0 
 12 7.6749E¯7 4.1119E¯25 9.9900E¯10 9.9754E¯2 2.0000E0 
 s: 7.6749E¯7 p: 9.9754E¯2 2.0000E0

### Gauss function

In [9]:
Gauss←{a s m c←⍺ ⋄ c+a×m s X ⍵} ⋄ X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)}
GaussEv←{x y←⍺ ⋄ a s m c←⍵ ⋄ x2←(x1←(xe←m s X x)×a×(x-m)÷×⍨s)×(x-m)÷s ⋄ (y-⍵ Gauss x)(⍉↑(-xe)x1(-x2)(-=⍨xe))}
p←10 5 1.5 2 ⋄ y←0 NOISY p Gauss x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Gauss x)(⊃x y GaussEv p)
(x y GaussEv LMA(8 4 1 1)(dini:1e¯3 ⋄ tols:1e¯6)).(iter sse rel dnorm p)
(x y GaussEv LMA(8 4 1 1)1e¯1(tols:1e¯6)).(iter sse rel dnorm p)
(x y GaussEv LMA(8 4.5 1.2 1.5)(tolr:1e¯3)).(iter sse rel dnorm p)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬──────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│1220703140│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴──────────┴───────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬─────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│122070314│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴─────────┴───────────────────────────────────────────────┘

┌─┬────────────┬───────────────┬──┬───────────────────────────────────────────────┐
│7│0.9170106519│0.0007211535235│25│9.471886955 4.249297788 2.168664717 2.526269831│
└─┴────────────┴───────────────┴──┴───────────────────────────────────────────────┘